- date        : 뉴스 작성 날짜
- title       : 뉴스 제목
- content     : 뉴스 본문
- keyword     : 검색 키워드
- industry    : 산업명 (반도체 / 디스플레이 / 이차전지) 

# 라이브러리

In [ ]:
!pip install selenium webdriver-manager

# 크롤링

## Chrome 창 열기

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# ===============================
# Chrome 옵션 설정
# ===============================
options = Options()

# 🔕 브라우저 권한 팝업 차단 (가장 중요)
prefs = {
    "profile.default_content_setting_values.notifications": 2,  # 알림 차단
    "profile.default_content_setting_values.geolocation": 2,     # 위치 차단
    "profile.default_content_setting_values.media_stream": 2     # 카메라/마이크 차단
}
options.add_experimental_option("prefs", prefs)

# 🔧 안정성 옵션
options.add_argument("--disable-notifications")
options.add_argument("--disable-infobars")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--start-maximized")

# ===============================
# 드라이버 생성 (단 한 번)
# ===============================
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)


## 회사, 키워드 dic 생성
- 최종 keyword 에 [회사+키워드] 조합 사용하기 위해서 

In [ ]:
SEMICONDUCTOR = {
    "industry": "반도체",
    "companies": [
        "마이크로소프트",
        "아마존",
        "구글",
        "엔비디아",
        "메타"
    ],
    "keywords": [
        "AI 서버 투자",
        "데이터센터 증설",
        "CAPEX 확대",
        "반도체 설비투자",
        "파운드리 증설",
        "AI 반도체 수요",
        "서버 수요 증가",
        "데이터센터 투자 계획",
        "반도체 발주",
        "공정 투자 확대"
    ]
}

DISPLAY = {
    "industry": "디스플레이",
    "companies": [
        "애플",
        "삼성전자",
        "샤오미",
        "오포",
        
        "비보"
    ],
    "keywords": [
        "OLED 채택",
        "디스플레이 사양 변경",
        "신제품 출시",
        "OLED 전환",
        "패널 채용",
        "고해상도 디스플레이",
        "저전력 디스플레이",
        "스마트폰 라인업 변경",
        "패널 수요 증가",
        "OLED 물량 확대"
    ]
}

BATTERY = {
    "industry": "이차전지",
    "companies": [
        "테슬라",
        "현대자동차",
        "폭스바겐",
        "BYD",
        "GM"
    ],
    "keywords": [
        "EV 생산 확대",
        "전기차 판매 둔화",
        "EV 생산 목표",
        "배터리 계약",
        "전기차 출하량",
        "EV 라인 증설",
        "배터리 수요 증가",
        "전기차 감산",
        "EV 정책 변화",
        "전기차 보조금"
    ]
}



In [ ]:
INDUSTRY_CONFIGS = [SEMICONDUCTOR, DISPLAY, BATTERY]

# =====================================================
# 📌 search_queries 생성 (🔥 job 단위)
# =====================================================
search_queries = []

for config in INDUSTRY_CONFIGS:
    industry = config["industry"]

    for company in config["companies"]:
        for keyword in config["keywords"]:
            search_queries.append({
                "industry": industry,
                "company": company,
                "event_keyword": keyword,
                "query": f"{company} {keyword}"
            })

print(f"✅ 생성된 job 수: {len(search_queries)}")

## base url 설정 및 기간(23년 12월) 지정

In [ ]:
from urllib.parse import quote_plus

BASE_URL = "https://search.naver.com/search.naver"
start_date = "2023.12.01"
end_date = "2023.12.31"

def make_search_url(query):
    encoded = quote_plus(query)
    return (
        f"{BASE_URL}"
        f"?where=news"
        f"&query={encoded}"
        f"&pd=3"
        f"&ds={start_date}"
        f"&de={end_date}"
        f"&sort=1"
    )


## 기사 후보 수 판단
- 이 코드에서는 뭘 수집하거나 그러진 않고 키워드에 따른 기사가 몇 개인지만 확인
- 기존에 스크롤링 코드가 추가되어있지 않아서 10개가 max 기사 개수로 오해 ... -> 스크롤 코드 추가

In [ ]:
import time
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# ===============================
# 스크롤 함수
# ===============================
def scroll_until_no_more(driver, pause_time=2, max_scroll=20):
    """
    더 이상 스크롤로 내용이 늘어나지 않을 때까지 하단 스크롤
    """
    last_height = driver.execute_script(
        "return document.body.scrollHeight"
    )

    for _ in range(max_scroll):
        driver.execute_script(
            "window.scrollTo(0, document.body.scrollHeight);"
        )
        time.sleep(pause_time)

        new_height = driver.execute_script(
            "return document.body.scrollHeight"
        )

        if new_height == last_height:
            break

        last_height = new_height


### 전체 뉴스 기사
- 기사 후보 수만 확인 가능

In [ ]:
# ===============================
# 메인 크롤링
# ===============================
results = []

for item in search_queries:
    query = item["query"]
    print(f"\n🔍 검색어: {query}")

    driver.get(make_search_url(query))

    try:
        # 검색 결과 영역 로딩 대기
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'div.fds-news-item-list-tab')
            )
        )

    except TimeoutException:
        # 검색 결과 없음
        print("❌ 검색결과가 없습니다.")
        results.append({
            "industry": item["industry"],
            "company": item["company"],
            "event_keyword": item["event_keyword"],
            "query": query,
            "has_result": False
        })
        continue

    # 🔥 스크롤 추가 (이게 핵심)
    scroll_until_no_more(driver)

    # 전체 기사 블록 다시 수집
    article_blocks = driver.find_elements(
        By.CSS_SELECTOR,
        'div.fds-news-item-list-tab > div'
    )

    print("📰 기사 후보 수:", len(article_blocks))



In [ ]:
for article in article_blocks:
    links = article.find_elements(
        By.CSS_SELECTOR,
        'a[data-heatmap-target=".tit"]'
    )
    if not links:
        continue

    title_link = links[0]
    article_url = title_link.get_attribute("href")

    spans = article.find_elements(
        By.CSS_SELECTOR,
        'div.sds-comps-profile-info-subtexts span'
    )
    date = spans[1].text.strip() if len(spans) > 1 else None

    print("✔", date, title_link.text)

    results.append({
        "industry": item["industry"],
        "company": item["company"],
        "event_keyword": item["event_keyword"],
        "query": query,
        "has_result": True,
        "date": date,
        "short_title": title_link.text,
        "url": article_url
    })

    # break  # 🔹 검색어당 1개만 (전체 수집하려면 제거)


In [ ]:
no_result_count = sum(
    1 for r in results if r.get("has_result") is False
)

print(f"❌ 검색결과 없는 쿼리 수: {no_result_count}")


## 언론사별 selector 지정
- 사이트별로 제목과 본문을 담고 있는 selector 이 상이하기 때문에 설정

In [ ]:
MEDIA_SELECTORS = {
    # 아이티데일리
    'www.itdaily.kr':{
        "title": "h3.heading",
        "content": "div#article-view-content-div > p"
    },
    # 비즈니스포스트
    'www.businesspost.co.kr':{
        "title": "h2",
        # "content": "div.detail_editor > div",
        "content": "#center-box > div.container.sub2_detail2 > div:nth-child(1) > div.detail_box2 > div > div.detail_editor",
    },
    # 이코노미스트
    'economist.co.kr':{
        "title": "h3.line_clamp_2",
        "content": "div#article_body"
    },
    # 아시아타임즈
    'www.asiatime.co.kr':{
        "title": "h6",
        "content": "div#article-view-content-div > p"
    },

    # 디지털투데이
    'www.digitaltoday.co.kr':{
        "title": "h3.heading",
        "content": "div#article-view-content-div > p"
    },

    # 주간동아
    'weekly.donga.com':{
        "title": "p.title_text",
        "content": "div.article_view"
    },

    # 디지털데일리
    'www.ddaily.co.kr':{
        "title": "p.title",
        "content": "div.article_content"
    },

    "www.itbiznews.com": {
        "title": "h1.heading",
        "content": "div#article-view-content-div"
    },
    "www.yna.co.kr": {
        "title": "h1.tit",
        "content": "div#articleWrap div.article"
    },
    "www.mk.co.kr": {
        "title": "h1.top_title",
        "content": "div#article_body"
    },
    "news.mt.co.kr": {
        "title": "h1.tit",
        "content": "div#textBody"
    },
    "www.etnews.com": {
        "title": "h1.title",
        "content": "div#articleBody"
    },
    "www.sedaily.com": {
        "title": "h1.art_tit",
        "content": "div#articletxt"
    },
    "biz.chosun.com": {
        "title": "h1.news-title",
        "content": "div#news-body"
    },
    "www.hankyung.com": {
        "title": "h1.headline",
        "content": "div#articletxt"
    },
    "zdnet.co.kr": {
        "title": "h1.sub_title",
        "content": "div#content"
    },
    "www.dt.co.kr": {
        "title": "h1.title",
        "content": "div#articleBody"
    },
    "www.sisajournal-e.com": {
        "title": "h1.heading",
        "content": "article.article-view-content-div > p"
    },
    "www.newspim.com": {
        "title": "h1#main-title",
        "content": "div#news-contents > p"
    },
    "www.sisajournal-e.com": {
        "title": "h1.heading",
        "content": "article.article-view-content-div > p"
    },
    "www.sisajournal-e.com": {
        "title": "h1.heading",
        "content": "article.article-view-content-div > p"
    },
}

FALLBACK_TITLE = "h1"
FALLBACK_CONTENT = "p"


# 테스트

In [ ]:
# ===============================
# 🔍 테스트할 검색어 하나만 선택
# ===============================
test_item = search_queries[0]   # ← 원하는 인덱스로 바꿔도 됨
query = test_item["query"]

print(f"\n🔍 [TEST] 검색어: {query}")

driver.get(make_search_url(query))

try:
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, 'div.fds-news-item-list-tab')
        )
    )
except TimeoutException:
    print("❌ 검색결과가 없습니다.")
else:
    # 🔥 스크롤 테스트
    scroll_until_no_more(driver, pause_time=2, max_scroll=10)

    article_blocks = driver.find_elements(
        By.CSS_SELECTOR,
        'div.fds-news-item-list-tab > div'
    )

    print("📰 [TEST] 기사 후보 수:", len(article_blocks))


In [ ]:
for i, article in enumerate(article_blocks[:5], 1):  # 테스트용 5개만
    try:
        # 제목
        title_el = article.find_element(
            By.CSS_SELECTOR,
            'a[data-heatmap-target=".tit"]'
        )
        title = title_el.text.strip()

        # 날짜
        spans = article.find_elements(
            By.CSS_SELECTOR,
            'div.sds-comps-profile-info-subtexts span'
        )
        date = spans[1].text.strip() if len(spans) > 1 else None

        print(f"{i}. [{date}] {title}")

    except Exception as e:
        print(f"{i}. ❌ 추출 실패:", e)


In [ ]:
from selenium.webdriver.common.by import By

def extract_text_safe(driver, selector):
    try:
        elements = driver.find_elements(By.CSS_SELECTOR, selector)
        text = " ".join(
            el.text.strip() for el in elements if el.text.strip()
        )
        return text if text else None
    except:
        return None


In [ ]:
from urllib.parse import urlparse

def extract_article_with_media_selectors(driver):
    """
    이미 기사 페이지에 들어와 있다고 가정
    """
    current_url = driver.current_url
    domain = urlparse(current_url).netloc

    selectors = MEDIA_SELECTORS.get(domain, {})

    title_candidates = [
        selectors.get("title"),
        FALLBACK_TITLE
    ]
    content_candidates = [
        # selectors.get("content"),
        selectors.get("body"),
        FALLBACK_CONTENT
    ]

    title, content = None, None

    # ----- 제목 -----
    for sel in title_candidates:
        if sel:
            title = extract_text_safe(driver, sel)
            if title:
                break

    # ----- 본문 -----
    for sel in content_candidates:
        if sel:
            content = extract_text_safe(driver, sel)
            if content:
                break

    return {
        "media": domain,
        "title": title,
        "content": content,
        "url": current_url
    }


In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

results = []

article_blocks = driver.find_elements(
    By.CSS_SELECTOR,
    'div.fds-news-item-list-tab > div'
)

print(f"📰 기사 후보 수: {len(article_blocks)}")

for i in range(len(article_blocks)):
    try:
        # ⚠️ 뒤로 오면 DOM이 재생성되므로 매번 다시 찾기
        article_blocks = driver.find_elements(
            By.CSS_SELECTOR,
            'div.fds-news-item-list-tab > div'
        )
        article = article_blocks[i]

        # 🔹 검색 결과 페이지의 제목 링크
        title_el = article.find_element(
            By.CSS_SELECTOR,
            'a[data-heatmap-target=".tit"]'
        )

        # 🔹 href 추출
        article_url = title_el.get_attribute("href")
        print(f"\n[{i+1}] 기사 이동:", article_url)

        # 🔹 클릭 ❌ / URL 직접 이동 ⭕
        driver.get(article_url)

        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )

        # 🔹 MEDIA_SELECTORS + fallback 적용 수집
        article_data = extract_article_with_media_selectors(driver)

        print("   언론사:", article_data["media"])
        print("   제목:", article_data["title"][:] if article_data["title"] else "❌ 제목 실패")
        print("   본문 길이:", len(article_data["content"]) if article_data["content"] else 0)
        print("\n[본문내용]")
        print("=" * 70)
        print(article_data["content"] if article_data["content"] else "본문 추출 실패")
        print("=" * 70)

        results.append(article_data)

    except Exception as e:
        print(f"⚠️ [{i+1}] 실패:", e)

    finally:
        # 🔹 검색 결과 페이지로 복귀
        driver.back()
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'div.fds-news-item-list-tab')
            )
        )
        time.sleep(0.5)


In [ ]:
import pandas as pd
import os

# ================================
# 📌 CSV 저장 경로
# ================================
SAVE_PATH = "news_results.csv"

# ================================
# 📌 results → DataFrame
# ================================
df_new = pd.DataFrame(results)

# 컬럼 정리 (없으면 None으로 채움)
expected_cols = ["media", "title", "content", "url"]
for col in expected_cols:
    if col not in df_new.columns:
        df_new[col] = None

df_new = df_new[expected_cols]

# ================================
# 📌 기존 CSV 있으면 이어쓰기
# ================================
if os.path.exists(SAVE_PATH):
    try:
        df_old = pd.read_csv(SAVE_PATH)
        df_final = pd.concat([df_old, df_new], ignore_index=True)
        print(f"📂 기존 CSV 로드: {len(df_old)}건")
    except Exception as e:
        print("⚠️ 기존 CSV 읽기 실패 → 새로 저장", e)
        df_final = df_new
else:
    df_final = df_new
    print("🆕 신규 CSV 생성")

# ================================
# 📌 CSV 저장
# ================================
df_final.to_csv(
    SAVE_PATH,
    index=False,
    encoding="utf-8-sig"
)

print(f"💾 CSV 저장 완료: {SAVE_PATH} (총 {len(df_final)}건)")


## 전체

#### 시도 3

In [ ]:
import time
import os
import pandas as pd
from urllib.parse import urlparse, quote_plus
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ===============================
# Chrome 옵션 설정
# ===============================
options = Options()

# 🔕 브라우저 권한 팝업 차단 (가장 중요)
prefs = {
    "profile.default_content_setting_values.notifications": 2,  # 알림 차단
    "profile.default_content_setting_values.popups": 2,         # 팝업 차단 (브라우저 수준)
    "profile.default_content_setting_values.geolocation": 2,     # 위치 차단
    "profile.default_content_setting_values.media_stream": 2,     # 카메라/마이크 차단
    "profile.managed_default_content_settings.images": 1,        # 이미지 (0으로 하면 속도는 빠르나 일부 사이트 오류 가능)

}
options.add_experimental_option("prefs", prefs)





# =====================================================
# 📌 1. 설정 및 경로
# =====================================================
SAVE_PATH = "industry_news_ver1.csv"
FAILED_PATH = "failed_urls.csv"  # 실패한 기사들만 모으는 파일

START_DATE = "2023.12.01"
END_DATE = "2023.12.31"

# 언론사별 셀렉터 설정
MEDIA_SELECTORS = {
    "www.itdaily.kr": {"title": "h3.heading", "content": "div#article-view-content-div > p"},
    "www.businesspost.co.kr": {"title": "h2", "content": "div.detail_editor"},
    "economist.co.kr": {"title": "h3.line_clamp_2", "content": "div#article_body"},
    "www.asiatime.co.kr": {"title": "h6", "content": "div#article-view-content-div > p"},
    "www.digitaltoday.co.kr": {"title": "h3.heading", "content": "div#article-view-content-div > p"},
    "weekly.donga.com": {"title": "p.title_text", "content": "div.article_view"},
    "www.ddaily.co.kr": {"title": "p.title", "content": "div.article_content"},
    "www.itbiznews.com": {"title": "h1.heading", "content": "div#article-view-content-div"},
    "www.yna.co.kr": {"title": "h1.tit", "content": "div#articleWrap div.article"},
    "www.mk.co.kr": {"title": "h1.top_title", "content": "div#article_body"},
    "news.mt.co.kr": {"title": "h1.tit", "content": "div#textBody"},
    "www.etnews.com": {"title": "h1.title", "content": "div#articleBody"},
    "www.sedaily.com": {"title": "h1.art_tit", "content": "div#articletxt"},
    # 조선비즈
    "biz.chosun.com": {"title": "h1.article-header__headline > span", "content": "secftion.article-body > p"},
    "www.hankyung.com": {"title": "h1.headline", "content": "div#articletxt"},
    "zdnet.co.kr": {"title": "h1.sub_title", "content": "div#content"},
    "www.dt.co.kr": {"title": "h1.title", "content": "div#articleBody"},
    "www.newspim.com": {"title": "h1#main-title", "content": "div#news-contents > p"},
    # 이데일리
    "www.edaily.co.kr": {"title": "h1", "content": "div.news_body"},
    # 비스니스포스트
    "www.businesspost.co.kr": {"title": "h2", "content": "div#detail_tab_cont"},
    # 동아일보
    "": {"title": "h1", "content": "div.main_view"},
    # cnb news
    "www.cnbnews.com": {"title": "h2.article_title", "content": "div#news_body_area > p "},
    # ilyo 
    "www.ilyo.co.kr": {"title": "h1.atcTitle", "content": "div.contentView"},
    # 언론사
    "": {"title": "", "content": ""},

}

FALLBACK_TITLE = "h1"
FALLBACK_CONTENT = "article p, div.article_body, div#articleBody, .news_article"

# =====================================================
# 📌 2. 산업별 키워드 설정 및 Job 생성
# =====================================================
INDUSTRY_CONFIGS = [
    {"industry": "반도체", "companies": ["마이크로소프트", "아마존", "구글", "엔비디아", "메타"], 
     "keywords": ["AI 서버 투자", "데이터센터 증설", "CAPEX 확대", "반도체 설비투자", "AI 반도체 수요"]},
    {"industry": "디스플레이", "companies": ["애플", "삼성전자", "샤오미", "오포", "비보"], 
     "keywords": ["OLED 채택", "디스플레이 사양 변경", "신제품 출시", "OLED 전환", "패널 수요 증가"]},
    {"industry": "이차전지", "companies": ["테슬라", "현대자동차", "폭스바겐", "BYD", "GM"], 
     "keywords": ["EV 생산 확대", "전기차 판매 둔화", "배터리 계약", "전기차 출하량", "전기차 보조금"]}
]

search_queries = []
for config in INDUSTRY_CONFIGS:
    for company in config["companies"]:
        for keyword in config["keywords"]:
            search_queries.append({
                "industry": config["industry"], "company": company, "event_keyword": keyword, "query": f"{company} {keyword}"
            })

# =====================================================
# 📌 3. 유틸리티 함수 (스크롤 함수 포함)
# =====================================================
def init_driver():
    options = Options()
    options.add_experimental_option("prefs", {"profile.default_content_setting_values.notifications": 2})
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--disable-notifications")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-popup-blocking")           # 팝업 차단 활성화


    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.set_page_load_timeout(30)
    return driver

def scroll_until_no_more(driver, pause_time=1.5, max_scroll=30):
    """검색 결과 페이지를 끝까지 스크롤하여 모든 기사 로드"""
    print("⏳ 모든 기사를 불러오기 위해 스크롤을 내리는 중...", end="", flush=True)
    last_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(max_scroll):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        print(".", end="", flush=True)
    print(" 완료!")

def make_search_url(query):
    encoded = quote_plus(query)
    return (f"https://search.naver.com/search.naver?where=news&query={encoded}"
            f"&pd=3&ds={START_DATE}&de={END_DATE}&sort=1")

def extract_text_safe(driver, selector):
    try:
        els = driver.find_elements(By.CSS_SELECTOR, selector)
        texts = [e.text.strip() for e in els if e.text.strip()]
        return "\n".join(texts) if texts else None
    except: return None

def extract_article(driver):
    domain = urlparse(driver.current_url).netloc
    selectors = MEDIA_SELECTORS.get(domain)
    title, content = None, None

    if selectors:
        title = extract_text_safe(driver, selectors["title"])
        content = extract_text_safe(driver, selectors["content"])

    if not title or not content:
        f_title, f_content = extract_text_safe(driver, FALLBACK_TITLE), extract_text_safe(driver, FALLBACK_CONTENT)
        if not title: title = f_title
        if not content: content = f_content

    if title and content:
        return {"status": "SUCCESS", "title": title, "content": content}
    return {"status": "FAIL", "title": title, "content": content}

# =====================================================
# 📌 4. 기존 데이터 로드 (Resume 기능)
# =====================================================
results, visited_urls, completed_jobs = [], set(), set()
article_id = 1

if os.path.exists(SAVE_PATH):
    try:
        df_existing = pd.read_csv(SAVE_PATH)
        if not df_existing.empty:
            results = df_existing.to_dict("records")
            visited_urls = set(df_existing["url"].dropna().astype(str))
            article_id = int(df_existing["ID"].max()) + 1
            completed_jobs = set(df_existing.groupby(["industry", "company", "event_keyword"]).size().index.tolist())
            print(f"🔁 기존 데이터 로드: {len(results)}건 | {len(completed_jobs)}개 Job 건너뜀")
    except Exception as e: print(f"⚠️ 기존 파일 로드 오류: {e}")

# =====================================================
# 🚀 5. 메인 크롤링 실행
# =====================================================
driver = init_driver()
main_window = driver.current_window_handle # 메인 검색창 핸들 고정

try:
    for item in search_queries:
        job_key = (item["industry"], item["company"], item["event_keyword"])
        if job_key in completed_jobs: continue

        print(f"\n🔥 JOB 시작: {job_key}")
        try:
            driver.get(make_search_url(item["query"]))
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            
            # [추가됨] 스크롤을 끝까지 내려서 모든 기사 로드
            scroll_until_no_more(driver)
            
            # 모든 URL 수집
            links = driver.find_elements(By.CSS_SELECTOR, 'a.news_tit, a[data-heatmap-target=".tit"]')
            article_urls = [l.get_attribute("href") for l in links if l.get_attribute("href") and l.get_attribute("href") not in visited_urls]
            print(f"🔗 총 {len(article_urls)}개의 기사 URL을 확보했습니다.")

###########################
# 기사 개별 추출 시작
            for url in article_urls:
                try:
                    # 1. 기사 페이지 접속 전, 불필요한 새 창들 모두 정리
                    for handle in driver.window_handles:
                        if handle != main_window:
                            driver.switch_to.window(handle)
                            driver.close()
                    driver.switch_to.window(main_window) # 메인창(검색결과)으로 일단 복귀

                    # 2. 기사 페이지 접속
                    driver.get(url)
                    
                    # 3. 로딩 대기 및 광고 제거
                    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
                    driver.execute_script("""
                        const ads = document.querySelectorAll('div[class*="ad"], div[id*="ad"], iframe, .popup, #popup');
                        ads.forEach(ad => ad.remove());
                    """)

                    # 4. 데이터 추출 (딱 한 번만 실행)
                    res = extract_article(driver)
                    
                    # 5. 추출 결과에 따른 출력 및 저장
                    if res["status"] == "SUCCESS":
                        print(f"\n[{article_id}] : 기사 수집 성공")
                        print(f"- 제목: {res['title']}")
                        print(f"- 본문길이 : {len(res['content'])}")
                        print(f"- 본문내용 (50자까지만) : {res['content'][:50].replace(chr(10), ' ')}...")
                        print(f"- 출처(url) : {url}")
                        
                        results.append({
                            "ID": article_id, 
                            "industry": item["industry"], 
                            "company": item["company"],
                            "event_keyword": item["event_keyword"], 
                            "뉴스 제목": res['title'],
                            "뉴스 본문": res['content'], 
                            "키워드": item["query"], 
                            "url": url
                        })
                        visited_urls.add(url)
                        article_id += 1
#                     else:
#                         # 실패 시에도 출력이 나오도록 설정
#                         print(f"\n[{article_id}] : 기사 수집 실패")
#                         if not res.get('title') and not res.get('content'):
#                             print("- 제목 및 본문 추출 실패 (둘 다 실패)")
#                         elif not res.get('title'):
#                             print("- 제목 추출 실패")
#                         else:
#                             print("- 본문 추출 실패")
#                         print(f"- 출처(url) : {url}")
                    
#                     print("-" * 60)

#                 except Exception as e:
#                     print(f"\n[{article_id}] : 처리 중 에러 발생 ({type(e).__name__})")
#                     print(f"- 에러 내용: {str(e)}")
#                     continue # 에러 발생 시 다음 기사로 진행

#             completed_jobs.add(job_key)
#             pd.DataFrame(results).to_csv(SAVE_PATH, index=False, encoding="utf-8-sig")
#             print(f"💾 JOB 저장 완료: {item['query']}")
#         except Exception as e: print(f"🚨 JOB 치명적 에러: {e}"); continue
# finally:
#     driver.quit()
#     print("\n🎉 모든 수집 프로세스가 종료되었습니다.")
                    else:
                        # ❌ 실패 출력 및 별도 CSV 저장
                        print(f"\n[{article_id}] : 기사 수집 실패")
                        reason = "둘 다 실패" if not res.get('title') and not res.get('content') else "제목 실패" if not res.get('title') else "본문 실패"
                        print(f"- {reason}\n- 출처(url) : {url}")

                        failed_data = {
                            "industry": item["industry"], "company": item["company"],
                            "event_keyword": item["event_keyword"], "query": item["query"],
                            "url": url, "reason": reason
                        }
                        # mode='a'를 사용하여 실패할 때마다 파일 끝에 추가
                        pd.DataFrame([failed_data]).to_csv(FAILED_PATH, index=False, mode='a', header=not os.path.exists(FAILED_PATH), encoding="utf-8-sig")

                    print("-" * 60)

                except Exception as e:
                    print(f"\n[{article_id}] : 에러 발생 ({type(e).__name__})")
                    continue

            completed_jobs.add(job_key)
            pd.DataFrame(results).to_csv(SAVE_PATH, index=False, encoding="utf-8-sig")
            print(f"💾 JOB 저장 완료: {item['query']}")
        except Exception as e: print(f"🚨 JOB 치명적 에러: {e}"); continue
finally:
    driver.quit()
    print("\n🎉 모든 수집 프로세스가 종료되었습니다.")

# 빅카인즈 크롤링 
- 키워드 선정 후에 기간 설정 후 데이터(엑셀 다운로드) 가져오는 자동화 코드

## 0. Chrome 창 열기

---
- 다운로드 경로의 경우 일반적인 경우와는 다르게 `ChromeOptions` 를 사용해줘야 함
---

In [ ]:
# =========================
# 홈페이지 열기 + 최대화
# =========================
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.action_chains import ActionChains

DOWNLOAD_DIR = r"C:\Users\82104\Desktop\Growth-Company-Prediction\industry_news\news_data"

prefs = {
    "download.default_directory": DOWNLOAD_DIR,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}

options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", prefs)


driver = webdriver.Chrome(options=options)
driver.get("https://www.bigkinds.or.kr/")
driver.maximize_window()   



## 0-1. 로그인 상태로 만들어주기

---
- 따로 key를 설정하고 싶지 않아서 로그인 페이지에서 아이디, 비밀번호는 직접 입력할 수 있도록 설정
---

In [ ]:
# =========================
# 로그인 버튼 클릭
# =========================
wait = WebDriverWait(driver, 15)
login_btn = wait.until(
    EC.element_to_be_clickable((
        By.CSS_SELECTOR,
        "#header > div.header-anim > div > div > div.topRightArea > ul > li > a"
    ))
)
login_btn.click()

# =========================
# 로그인 메뉴 클릭
# =========================
login_menu_btn = wait.until(
    EC.element_to_be_clickable((
        By.CSS_SELECTOR,
        "#header > div.header-anim > div > div > div.topRightArea > ul > li > div > ul > li > a"
    ))
)
login_menu_btn.click()

# =========================
# 네이버 로그인 클릭
# =========================
naver_login_btn = wait.until(
    EC.element_to_be_clickable((
        By.CSS_SELECTOR,
        "#login-modal > div > div > div > div.btn-sns "
        "> button.btn.btn-lg.btn-round.btn-naver.oauth-login-btn > i"
    ))
)
naver_login_btn.click()

print("🔐 네이버 로그인 화면으로 이동했습니다.")
print("👉 아이디 / 비밀번호 로그인 후 다시 돌아오면 자동 진행됩니다.")

# =========================
# ⚠️ 사람이 로그인할 시간 대기
# =========================
input("✅ 로그인 완료 후 ENTER 를 눌러주세요")


---
디버깅
- 새로고침 누르면 이전 화면으로 넘어가지 않음 
- 되긴 하는데 첫 번째 키워드인 반도체 전망은 여전히 안되지만 다른 것들은 문제 없이 됨
- 해당 selector 을 누르기는 하지만 다운은 안됨 

---
해결
- BIGKINDS 로고를 누를 수 있도록 변경해서 기간 설정부터 다시 하도록 설정
- 엑셀 파일 다운로드 실제로 되었는지 확인하는 코드 추가 
- 엑셀 파일도 변경할 수 있는 코드 추가
---

##### 1. 검색어 => 키워드가 똑같은 게 있으면 순회 x

In [ ]:
keywords_dict = {
    "keyword1": "반도체 전망",
    "keyword2": "디스플레이 전망",
    "keyword3": "이차전지 전망",
    "keyword4": "삼성 전망"
}
search_words = list(dict.fromkeys(keywords_dict.values()))
print("✅ 검색어 목록:", search_words)


##### 2. 대기 설정

In [ ]:
wait = WebDriverWait(driver, 15)
actions = ActionChains(driver)
print("✅ WebDriverWait / ActionChains 설정 완료")

##### 3. 기간 설정 함수

In [ ]:
def set_period():
    print("🗓️ [STEP] 기간 설정 시작")

    print("  - 기간 버튼 대기 중...")
    period_btn = wait.until(
        EC.element_to_be_clickable((By.ID, "ig-sd-btn"))
    )
    period_btn.click()
    print("  ✔ 기간 버튼 클릭")

    print("  - 시작일 입력 중...")
    begin_date = wait.until(
        EC.presence_of_element_located((By.ID, "search-begin-date"))
    )
    begin_date.send_keys(Keys.CONTROL + "a")
    begin_date.send_keys(Keys.DELETE)
    begin_date.send_keys("2023-12-01")
    print("  ✔ 시작일 입력 완료")

    print("  - 종료일 입력 중...")
    end_date = wait.until(
        EC.presence_of_element_located((By.ID, "search-end-date"))
    )
    end_date.send_keys(Keys.CONTROL + "a")
    end_date.send_keys(Keys.DELETE)
    end_date.send_keys("2023-12-31")
    print("  ✔ 종료일 입력 완료")

    print("  - 적용 버튼 대기 중...")
    apply_btn = wait.until(
        EC.element_to_be_clickable((
            By.CSS_SELECTOR,
            "#ds-modal button.apply-btn.primary-btn.news-search-btn"
        ))
    )
    apply_btn.click()
    print("  ✔ 기간 적용 버튼 클릭")

    time.sleep(2)
    print("🗓️ [STEP] 기간 설정 완료")

# set_period()


##### 4. 홈으로 이동 함수

In [ ]:
def go_home():
    print("🏠 홈 버튼 클릭 시도")

    home_btn = wait.until(
        EC.element_to_be_clickable((
            By.CSS_SELECTOR,
            "#header > div.header-anim > div > div > h1 > a > span"
        ))
    )
    home_btn.click()

    time.sleep(3)
    print("  ✔ 홈으로 이동 완료")

# go_home()

##### 5. 다운로드 완료 확인 함수

In [ ]:
import os
import time

def wait_for_excel_download(download_dir, timeout=60):
    """
    엑셀 파일 다운로드 완료 대기
    - 새로운 .xlsx 파일 생성
    - .crdownload 사라질 때까지 대기
    """
    print("  ⏳ 다운로드 완료 대기 중...")

    before_files = set(os.listdir(download_dir))
    start_time = time.time()

    while time.time() - start_time < timeout:
        current_files = set(os.listdir(download_dir))
        new_files = current_files - before_files

        excel_files = [f for f in new_files if f.endswith(".xlsx")]
        downloading = any(f.endswith(".crdownload") for f in current_files)

        if excel_files and not downloading:
            print(f"  📁 다운로드 완료 파일: {excel_files[0]}")
            return excel_files[0]

        time.sleep(1)

    print("  ❌ 다운로드 타임아웃")
    return None

##### 6. 파일명 변경 함수

In [ ]:
def rename_downloaded_file(download_dir, old_filename, new_filename):
    old_path = os.path.join(download_dir, old_filename)
    new_path = os.path.join(download_dir, new_filename)

    if os.path.exists(new_path):
        print(f"  ⚠️ 이미 존재: {new_filename} → 덮어쓰기 방지")
        return new_filename

    os.rename(old_path, new_path)
    print(f"  ✏️ 파일명 변경: {new_filename}")
    return new_filename

##### 7. 키워드 입력 - 엑셀 다운로드 - 5,6번 함수 실행해서 저장
- 감격의 눈물 .. 성공 .. ☃️

In [ ]:
# =========================
# 키워드 순회
# =========================
for idx, search_word in enumerate(search_words):
    print("\n==============================")
    print(f"🔍 [{idx+1}/{len(search_words)}] '{search_word}' 처리 시작")

    try:
        # 🔹 홈 → 기간 설정
        go_home()
        set_period()

        # STEP 1: 뉴스 검색
        wait.until(
            EC.element_to_be_clickable((By.ID, "collapse-step-1"))
        ).click()
        print("  ✔ 뉴스 검색 클릭")
        time.sleep(2)

        # STEP 2: 검색어 입력
        search_input = wait.until(
            EC.presence_of_element_located((By.ID, "total-search-key"))
        )
        search_input.clear()
        search_input.send_keys(search_word, Keys.ENTER)
        print("  ✔ 검색어 입력")
        time.sleep(3)

        # STEP 3: 분석 결과 버튼
        step3_btn = wait.until(
            EC.element_to_be_clickable((By.ID, "collapse-step-3"))
        )
        driver.execute_script(
            "arguments[0].scrollIntoView({block:'center'});", step3_btn
        )
        step3_btn.click()
        print("  ✔ 분석 결과 클릭")

        # 서버 워밍업
        wait_time = 10 if idx == 0 else 4
        print(f"  ⏳ 서버 워밍업 {wait_time}s")
        time.sleep(wait_time)

        # =========================
        # 경로 설정 + 파일명에 들어갈 변수 선언
        # =========================
        DOWNLOAD_DIR = r"C:\Users\82104\Desktop\Growth-Company-Prediction\industry_news\news_data"
        BEGIN_DATE = "20231201"
        END_DATE   = "20231231"

        # =========================
        # 엑셀 다운로드
        # =========================
        print("  ⬇️ 엑셀 다운로드 시도")
        clicked = False

        for attempt in range(10):
            buttons = driver.find_elements(
                By.XPATH, "//button[contains(., '엑셀 다운로드')]"
            )
            print(f"    🔁 시도 {attempt+1}/10 | 버튼 수: {len(buttons)}")

            for btn in buttons:
                if not btn.is_displayed() or not btn.is_enabled():
                    continue

                cls = btn.get_attribute("class") or ""
                if "disabled" in cls:
                    continue

                driver.execute_script(
                    "arguments[0].scrollIntoView({block:'center'});", btn
                )
                time.sleep(1)

                actions.move_to_element(btn).pause(1).perform()
                driver.execute_script("arguments[0].click();", btn)
                print("    ✔ 다운로드 클릭 성공")

                clicked = True
                break

            if clicked:
                break

            driver.execute_script("window.scrollBy(0, 400);")
            time.sleep(1)

        if not clicked:
            print(f"⚠️ '{search_word}' 다운로드 버튼 없음 → 스킵")
            continue

        print(f"✅ '{search_word}' 다운로드 요청 완료")

        # =========================
        # 🔥 다운로드 완료 확인
        # =========================
        downloaded_file = wait_for_excel_download(
            download_dir=DOWNLOAD_DIR,
            timeout=60
        )

        if downloaded_file is None:
            print(f"⚠️ '{search_word}' 엑셀 실제 다운로드 실패 → 다음 키워드")
            continue

        # =========================
        # 🔥 파일명 변경
        # =========================
        safe_keyword = search_word.replace(" ", "_")
        new_filename = f"{safe_keyword}_{BEGIN_DATE}_{END_DATE}.xlsx"

        rename_downloaded_file(
            download_dir=DOWNLOAD_DIR,
            old_filename=downloaded_file,
            new_filename=new_filename
        )

    except Exception as e:
        print(f"❌ '{search_word}' 처리 중 오류 발생")
        import traceback
        traceback.print_exc()
        continue

print("\n🎉 모든 키워드 처리 완료")


##### 8. 다운로드 경로 및 파일명 확인

In [ ]:
print("📂 DOWNLOAD_DIR 파일 목록:", os.listdir(DOWNLOAD_DIR))
print("📄 downloaded_file:", downloaded_file)


# 키워드별 concat

In [ ]:
import pandas as pd

In [ ]:
kw_1 = pd.read_excel('../data/디스플레이전망_NewsResult_20231201-20231231.xlsx')
kw_2 = pd.read_excel('../data/반도체전망_NewsResult_20231201-20231231.xlsx')
kw_3 = pd.read_excel('../data/이차전지전망_NewsResult_20231201-20231231.xlsx')
kw_3.head(1)

In [ ]:
import pandas as pd
import os

# =========================
# 1. 파일 경로 리스트
# =========================
file_paths = [
    '../data/디스플레이전망_NewsResult_20231201-20231231.xlsx',
    '../data/반도체전망_NewsResult_20231201-20231231.xlsx',
    '../data/이차전지전망_NewsResult_20231201-20231231.xlsx'
]

dfs = []

# =========================
# 2. 파일 순회
# =========================
for path in file_paths:
    # 엑셀 로드
    df = pd.read_excel(path)

    # 파일명만 추출
    filename = os.path.basename(path)

    # 검색어 추출 (_NewsResult 앞부분)
    search_word = filename.split('_NewsResult')[0]

    # 새 컬럼 추가
    df['검색어'] = search_word

    dfs.append(df)

# =========================
# 3. 데이터 합치기
# =========================
final_df = pd.concat(dfs, ignore_index=True)

final_df.head(1)


In [ ]:
final_df.to_csv('../output/키워드별concat.csv', encoding='utf-8-sig')